In [ ]:
from sympy import symbols, sqrt, Eq, solveset, lambdify, S, solve
x, y, z = symbols('x y z', real=True)
x_p, y_p, l, m, n, t = symbols('x_p y_p l m n t', real=True)
r = symbols('r', real=True)
c, k, R = symbols('c k R', real=True)
A, B, C, D, E, F, G, H, I, J = symbols('A B C D E F G H I J', real=True)

## Lens design formula: canonical quadric form

In [ ]:
## Lens design surface equation for 'Standard'.
eq_ld = Eq(z, c * r**2 / (1 + sqrt(1 - (1+k) * c**2 * r**2)))
eq_ld

In [ ]:
## General quadric formula
eq_quad = Eq(A*x**2 + B*y**2 + C*z**2 + D*x*y + E*y*z + \
      F*x*z + G*x + H*y + I*z + J, 0)
eq_quad

In [ ]:
## Lens design formula as canonical quadric
eq_ld_quad = eq_quad.subs([
    (A, 1), (B, 1),
    (C, (k+1)), (I, -2*R),
    (D,0), (E, 0), (F, 0), (G, 0), (H, 0), (J, 0)])
eq_ld_quad

## From general quadric to single sheet z= form

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Getting back to the original 'z =' form.
z_sols = solveset(eq_ld_quad, z).simplify()

## We have two sheets
z_1 = list(z_sols)[0]
z_2 = list(z_sols)[1]
display(z_1); display(z_2)

In [ ]:
## Plotting the two sheets
def zr1 (r, R, k):
    zr1 = (R + np.sqrt(R**2 - (k+1) * r**2)) / (k+1)
    return zr1

def zr2 (r, R, k):
    zr2 = (R - np.sqrt(R**2 - (k+1) * r**2)) / (k+1)
    return zr2

## The considered sheet can be either sheet 1 or sheet 2.
## We have to choose the one with z(r=0) = 0 in it
## on a case-by-case basis.

def zrsol (r, R, k):
    """Solution considered."""
    z = (R - np.sign(R) * np.sqrt(R**2 - (k+1) * r**2)) / (k+1)
    return z

R_val = -10
k_val = 3
r_val = np.linspace(-5, 5)
zp1 = zr1(r_val, R_val, k_val)
zp2 = zr2(r_val, R_val, k_val)
zpsol = zrsol(r_val, R_val, k_val)

plt.figure()
plt.plot(r_val, zp1, label='Sheet 1');
plt.plot(r_val, zp2, label='Sheet 2');
plt.plot(r_val, zpsol, label='Solution');
plt.legend(); plt.grid();

## Behaviour of the standard formula
We check the "smoothness" of the behaviour of the standard formula with
respect to parameter $k$.

In [ ]:
kvec = [-3, -2, -1.1, -1, -0.9, 0, 1, 2]
curv = 1 / 100

def z_std (r, k, c):
    """Standard altitude formula."""
    z = c * r**2 / (1 + np.sqrt(1 - (k+1) * r**2 * c**2))
    return z

rvec = np.linspace(-10, 10)
z_sphere = z_std(rvec, 0, curv)

plt.figure()
for kp in kvec:
    zvec = z_std(rvec, kp, curv) - z_sphere
    plt.plot(rvec, zvec, label='k=' + str(kp))
plt.grid(); plt.legend();

We see the behaviour is indeed smooth.

## Intersection with ray
We have a ray with expression
$$\begin{bmatrix}
x_p + t \cdot l \\
y_p + t \cdot m \\
t \cdot n
\end{bmatrix}$$

We want the intersection point with the standard surface.

In [ ]:
unit_eq = Eq(1, l**2 + m**2 + n**2)
unit_eq

### Using surface altitude expression

In [ ]:
## Standard surface expression
eq_ld_xyz = eq_ld.subs([(r**2, x**2 + y**2)])
eq_ld_xyz

In [ ]:
## We substitute the ray expression into the surface expression
eq_ld_ray = eq_ld_xyz.subs([(x, x_p + t*l),
                            (y, y_p + t*m),
                            (z, t*n)])
eq_ld_ray

In [ ]:
sols = solveset(eq_ld_ray, t, domain=S.Reals)

In [ ]:
sol1 = sols.args[1].args[2]
sol2 = sols.args[1].args[3]
display(sol1, sol2)

## The denominator is not super stable.

### Using general quadric form

In [ ]:
## Solving using the general quadric expression
eq_quad_ray = eq_ld_quad.subs([(x, x_p + t*l),
                               (y, y_p + t*m),
                               (z, t*n)])
eq_quad_ray

In [ ]:
sols_quad = solveset(eq_quad_ray, t, domain=S.Reals)
sols_quad.args[1]
## Denominator is still a problem (eg. for k = 0).

### Welford's solution

In [ ]:
WF = c * (x_p**2 + y_p**2)
WG = n - c * (l * x_p + m * y_p)
tsol = WF / (WG + sqrt(WG**2 - c * WF * (1 + k * n**2)))
display(tsol)
## This appears more numerically robust.

In [ ]:
## Check if this is indeed a solution
welford_check = eq_quad_ray.subs([(t, tsol), (R, 1/c)])
display(welford_check)
nval1 = sqrt(1 - l**2 - m**2)
nval2 = -sqrt(1 - l**2 - m**2)
welford_check1 = welford_check.subs([(n, nval1)])
welford_check2 = welford_check.subs([(n, nval2)])
display(welford_check1.simplify())
display(welford_check2.simplify())

In [ ]:
## Just to check we are on the right sheet, we check a sample with the surface equation.
welford_check_surf = eq_ld_ray.subs([(t, tsol), (R, 1/c)])
welford_check_surf = welford_check_surf.subs([(l, 0), (m, 0), (n, 1)])
display(welford_check_surf)